# Hyperparameter exploration


In [ ]:
import os

from ultralytics import YOLO
from ray import tune

def tune_model(prefix, pt_model, data, space):
    name = prefix + '_' + pt_model.split('.')[0] + '_' + data.split('.')[0]
    
    task = pt_model.split('-')[1].split('.')[0]
    if task == 'seg':
        task = 'segment'
    elif task == 'obb':
        task = 'OBB'
    else:
        raise ValueError('Invalid task type. Must be seg or obb')
    
    data_path = os.path.join(os.getcwd(), data)

    model = YOLO(pt_model, task)
    result_grid = model.tune(data=data_path, use_ray=True, space=space, grace_period=4, batch=-1, epochs=20, name=name, device=0, degrees=180, gpu_per_trial=1)
   
    return result_grid

# models = {'yolov8s-seg.pt': [640,0.3,0],'yolov9c-seg.pt': [640,0.3,0], 'yolov8s-obb.pt': [800,0.2,0.25]}
models = ['yolov8s-seg.pt', 'yolov9c-seg.pt', 'yolov8s-obb.pt']

datasets = ['cgi.yaml', 'human.yaml', 'combined.yaml']

space = {
    'lr0': tune.uniform(1e-6, 2e-1),
    'momentum': tune.uniform(0.6, 0.98),
    'box': tune.uniform(0.02, 10),
    'cls': tune.uniform(0.2, 4.0),
    'dropout': tune.uniform(0, 0.5)
}

for pt in models:
    for dataset in datasets:
        result = tune_model('tuner', pt, dataset, space)


# Check labels

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.transforms as transforms
from PIL import Image

# Load the image
image_path = 'datasets/rendered/train/bad_9.png'
# image_path = 'datasets/aspl_images/train/HHF22150120_40_10_56-MD_1.jpg'
image = Image.open(image_path)

image_width, image_height = image.size

# Load the label in YOLO polygon format
label_path = 'datasets/rendered/train/bad_9.txt'
# label_path = 'datasets/aspl_images/train/HHF22150120_40_10_56-MD_1.txt'
labels = []
with open(label_path, 'r') as f:
    labels = f.read().splitlines()

# Parse the label to extract the polygon coordinates
# Assuming the label is in the format: id x1 y1 x2 y2 x3 y3 x4 y4
shapes = {}
for label in labels:
    label = label.split(' ')
    label = [ float(x) for x in label ]
    point1 = (label[1]*image_width, label[2]*image_height)
    point2 = (label[3]*image_width, label[4]*image_height)
    point3 = (label[5]*image_width, label[6]*image_height)
    point4 = (label[7]*image_width, label[8]*image_height)

    print (point1, point2, point3, point4)
    shapes[label[0]] = [point1, point2, point3, point4]

# just plot the fucking points as corner of squares
# Plot the image
fig, ax = plt.subplots()
ax.imshow(image)
ax.axis('off')

for shape in shapes.values():
    polygon = patches.Polygon(shape, linewidth=1, edgecolor='r', facecolor='none')

    polygon.set_transform(ax.transData)

    # Add the patch to the Axes
    ax.add_patch(polygon)

plt.show()


# Training models

In [ ]:
import os

from ultralytics import YOLO

def train_model(pt_model, data, prefix, img_size, dropout):
    name = prefix + '_' + pt_model.split('.')[0] + '_' + data.split('.')[0]
    
    task = pt_model.split('-')[1].split('.')[0]
    if task == 'seg':
        task = 'segment'
    elif task == 'obb':
        task = 'obb'
    else:
        raise ValueError('Invalid task type. Must be seg or obb')

    # check if folder exists
    if os.path.exists(os.path.abspath(f"./runs/{task}/{name}")):
        return

    epochs = 100
    patience = 20

    batch_size = -1
    cache = True

    model = YOLO(pt_model)
    model.train(data=data, epochs=epochs, imgsz=img_size, patience=patience, batch=batch_size, cache=cache, name=name, dropout=dropout, device=0, degrees=180)
    return model

models = {
    'yolov8s-obb.pt': [640, 0.1],
    'yolov8s-seg.pt': [640, 0.05],
    'yolov9e-seg.pt': [640, 0.05],
}

datasets = ['cgi.yaml', 'human.yaml', 'combined.yaml']

prefix = 'control'

for model in models:
    for data in datasets:
        train_model(model, data, prefix, models[model][0], models[model][1])
